In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pickle



In [3]:
train_df = pd.read_csv("C:\\Users\\DELL\\Downloads\\kleit-ml-course-project-2025\\train.csv")


In [4]:
X = train_df.drop(columns=['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults'])
y = train_df[['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults']]


In [5]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)



In [6]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
model = MultiOutputClassifier(rf)
model.fit(X_train, y_train)



MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42))

In [7]:
y_pred = model.predict(X_val)
for i, col in enumerate(y.columns):
    print(f"--- {col} ---")
    print(classification_report(y_val[col], y_pred[:, i]))



--- Pastry ---
              precision    recall  f1-score   support

           0       0.92      1.00      0.96      3543
           1       0.52      0.04      0.07       301

    accuracy                           0.92      3844
   macro avg       0.72      0.52      0.52      3844
weighted avg       0.89      0.92      0.89      3844

--- Z_Scratch ---
              precision    recall  f1-score   support

           0       0.96      0.99      0.98      3624
           1       0.65      0.37      0.47       220

    accuracy                           0.95      3844
   macro avg       0.81      0.68      0.72      3844
weighted avg       0.94      0.95      0.95      3844

--- K_Scatch ---
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      3162
           1       0.91      0.90      0.90       682

    accuracy                           0.97      3844
   macro avg       0.94      0.94      0.94      3844
weighted avg       0.97 

In [8]:
with open('multi_output_rf_model.pkl', 'wb') as file:
    pickle.dump(model, file)
    print("Model saved as 'multi_output_rf_model.pkl'")

Model saved as 'multi_output_rf_model.pkl'


Model training done,generation of submission csv

In [9]:
from sklearn.multioutput import MultiOutputClassifier



In [10]:
with open('multi_output_rf_model.pkl', 'rb') as file:
    model = pickle.load(file)


In [11]:
test_df = pd.read_csv("C:\\Users\\DELL\\Downloads\\kleit-ml-course-project-2025\\test.csv")


In [12]:
print("Test data columns:", test_df.columns)
print("Test data shape:", test_df.shape)


Test data columns: Index(['id', 'X_Minimum', 'X_Maximum', 'Y_Minimum', 'Y_Maximum',
       'Pixels_Areas', 'X_Perimeter', 'Y_Perimeter', 'Sum_of_Luminosity',
       'Minimum_of_Luminosity', 'Maximum_of_Luminosity', 'Length_of_Conveyer',
       'TypeOfSteel_A300', 'TypeOfSteel_A400', 'Steel_Plate_Thickness',
       'Edges_Index', 'Empty_Index', 'Square_Index', 'Outside_X_Index',
       'Edges_X_Index', 'Edges_Y_Index', 'Outside_Global_Index', 'LogOfAreas',
       'Log_X_Index', 'Log_Y_Index', 'Orientation_Index', 'Luminosity_Index',
       'SigmoidOfAreas'],
      dtype='object')
Test data shape: (12814, 28)


In [14]:
sample_submission = pd.read_csv("C:\\Users\\DELL\\Downloads\\kleit-ml-course-project-2025\\sample_submission.csv")
print("Sample submission shape:", sample_submission.shape)


Sample submission shape: (12814, 8)


In [15]:
label_columns = ['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults']


In [16]:
X_test = test_df.drop(columns=[col for col in label_columns if col in test_df.columns])
print("Feature matrix shape:", X_test.shape)


Feature matrix shape: (12814, 28)


In [17]:
probabilities = pd.DataFrame(index=X_test.index)



In [18]:
if isinstance(model, MultiOutputClassifier):
    estimators = model.estimators_
    for i, col in enumerate(label_columns):
        probabilities[col] = estimators[i].predict_proba(X_test)[:, 1]
else:
    y_pred_prob = model.predict_proba(X_test)
    for i, col in enumerate(label_columns):
        probabilities[col] = y_pred_prob[i][:, 1]

print("Probabilities shape:", probabilities.shape)


Probabilities shape: (12814, 7)


In [20]:
if len(probabilities) != len(sample_submission):
    print(f"WARNING: Number of predictions ({len(probabilities)}) doesn't match sample submission ({len(sample_submission)})")

    id_col = None
    for col in sample_submission.columns:
        if col not in label_columns:
            id_col = col
            break
    
    if id_col and id_col in test_df.columns:
        print(f"Using {id_col} to align predictions with sample submission format")
        
        aligned_probabilities = pd.DataFrame(index=sample_submission.index)
        
       
        for col in label_columns:
            
            prob_map = dict(zip(test_df[id_col], probabilities[col]))
            aligned_probabilities[col] = sample_submission[id_col].map(prob_map)
        
        probabilities = aligned_probabilities


In [21]:
# Step 9: Ensure the order of columns in the submission matches the sample file
submission = sample_submission.copy()
for col in label_columns:
    if col in submission.columns:
        submission[col] = probabilities[col].values


In [22]:

# Step 10: Verify the submission format
print("Final submission shape:", submission.shape)
print("Submission columns:", submission.columns)


Final submission shape: (12814, 8)
Submission columns: Index(['id', 'Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps',
       'Other_Faults'],
      dtype='object')


In [23]:
submission.to_csv('submission.csv', index=False)
print("Submission file 'submission.csv' created successfully!")

Submission file 'submission.csv' created successfully!
